# Helper Functions

In [44]:
import json

import requests
import time


upload_endpoint = "https://api.assemblyai.com/v2/upload"
transcript_endpoint = "https://api.assemblyai.com/v2/transcript"

API_KEY = "a6f53d3fe7f542be8ce148432f0ff9a0"

## Make Header

In [2]:
def make_header(api_key):
    return {
    'authorization': api_key,
    'content-type': 'application/json'
    }

In [3]:
header = make_header(API_KEY)

In [4]:
header

{'authorization': 'a6f53d3fe7f542be8ce148432f0ff9a0',
 'content-type': 'application/json'}

## Upload File

In [5]:
# Helper for `upload_file()`
def _read_file(filename, chunk_size=5242880):
    with open(filename, "rb") as f:
        while True:
            data = f.read(chunk_size)
            if not data:
                break
            yield data


# Uploads a file to AAI servers
def upload_file(audio_file, header):
    upload_response = requests.post(
        upload_endpoint,
        headers=header, 
        data=_read_file(audio_file)
    )
    return upload_response#.json()

In [6]:
upload_response = upload_file("../s5_gettysburg.wav", header)

In [8]:
upload_response

<Response [200]>

In [12]:
for i in dir(upload_response):
    if not i.startswith("_"):
        print('-'*50)
        print(i, '---', getattr(upload_response, i))

--------------------------------------------------
apparent_encoding --- ascii
--------------------------------------------------
close --- <bound method Response.close of <Response [200]>>
--------------------------------------------------
connection --- <requests.adapters.HTTPAdapter object at 0x0000022703C387C0>
--------------------------------------------------
content --- b'{"upload_url": "https://cdn.assemblyai.com/upload/42bef839-18c7-4252-9fec-0ea53075d633"}'
--------------------------------------------------
cookies --- <RequestsCookieJar[<Cookie AWSALB=u4jxFFVYKbpQaexBa6qR3efyQJFaddP7dzurpb30gCysbRtebDRgYLfpjnP9UdQ8IapoqnVwTcFJB9llSr0KMEuIZnXXDGHcmhqfVze0iPBU/2llPTL5jx5Kow/+ for api.assemblyai.com/>, <Cookie AWSALBCORS=u4jxFFVYKbpQaexBa6qR3efyQJFaddP7dzurpb30gCysbRtebDRgYLfpjnP9UdQ8IapoqnVwTcFJB9llSr0KMEuIZnXXDGHcmhqfVze0iPBU/2llPTL5jx5Kow/+ for api.assemblyai.com/>]>
--------------------------------------------------
elapsed --- 0:00:00.875986
-------------------------------

In [13]:
upload_response.status_code == 200

True

In [14]:
upload_response.json()

{'upload_url': 'https://cdn.assemblyai.com/upload/42bef839-18c7-4252-9fec-0ea53075d633'}

In [16]:
# For restarting so dont have to re run
upload_response = 'https://cdn.assemblyai.com/upload/42bef839-18c7-4252-9fec-0ea53075d633'

## Request Transcript

In [17]:
def request_transcript(upload_url, header, **kwargs):
    # If input is a dict returned from `upload_file` rather than a raw upload_url string
    if type(upload_url) is dict:
        upload_url = upload_url['upload_url']

    # Create request
    transcript_request = {
        'audio_url': upload_url,
        **kwargs
    }

    # POST request
    transcript_response = requests.post(
        transcript_endpoint,
        json=transcript_request,
        headers=header
    )

    return transcript_response.json()

In [18]:
json_dict = {
    'redact_pii_policies': ['drug', 'injury', 'person_name'],
    'auto_chapters': 'true',
    'auto_highlights': 'true',
    'iab_categories': 'true',
    'entity_detection': 'true',
    'sentiment_analysis': 'true',
    'redact_pii': 'true',
    'content_safety': 'true',
}

In [19]:
transcript_request_response = request_transcript(upload_response, header, **json_dict)

In [20]:
transcript_request_response

{'id': 'owke4eku0d-8ac7-4729-bdb9-1a42fcf6f70b',
 'language_model': 'assemblyai_default',
 'acoustic_model': 'assemblyai_default',
 'language_code': 'en_us',
 'status': 'queued',
 'audio_url': 'https://cdn.assemblyai.com/upload/42bef839-18c7-4252-9fec-0ea53075d633',
 'text': None,
 'words': None,
 'utterances': None,
 'confidence': None,
 'audio_duration': None,
 'punctuate': True,
 'format_text': True,
 'dual_channel': None,
 'webhook_url': None,
 'webhook_status_code': None,
 'webhook_auth': False,
 'webhook_auth_header_name': None,
 'speed_boost': False,
 'auto_highlights_result': None,
 'auto_highlights': True,
 'audio_start_from': None,
 'audio_end_at': None,
 'word_boost': [],
 'boost_param': None,
 'filter_profanity': False,
 'redact_pii': True,
 'redact_pii_audio': False,
 'redact_pii_audio_quality': 'mp3',
 'redact_pii_policies': ['drug', 'injury', 'person_name'],
 'redact_pii_sub': 'hash',
 'speaker_labels': False,
 'content_safety': True,
 'iab_categories': True,
 'content_s

In [24]:
for i in dir(transcript_request_response):
    if not i.startswith("_"):
        print('-'*50)
        print(i, '---', getattr(transcript_request_response, i))

--------------------------------------------------
clear --- <built-in method clear of dict object at 0x0000022713DCC300>
--------------------------------------------------
copy --- <built-in method copy of dict object at 0x0000022713DCC300>
--------------------------------------------------
fromkeys --- <built-in method fromkeys of type object at 0x00007FFFCDDBEBB0>
--------------------------------------------------
get --- <built-in method get of dict object at 0x0000022713DCC300>
--------------------------------------------------
items --- <built-in method items of dict object at 0x0000022713DCC300>
--------------------------------------------------
keys --- <built-in method keys of dict object at 0x0000022713DCC300>
--------------------------------------------------
pop --- <built-in method pop of dict object at 0x0000022713DCC300>
--------------------------------------------------
popitem --- <built-in method popitem of dict object at 0x0000022713DCC300>
--------------------------

In [28]:
print(transcript_request_response)

{'id': 'owke4eku0d-8ac7-4729-bdb9-1a42fcf6f70b', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'queued', 'audio_url': 'https://cdn.assemblyai.com/upload/42bef839-18c7-4252-9fec-0ea53075d633', 'text': None, 'words': None, 'utterances': None, 'confidence': None, 'audio_duration': None, 'punctuate': True, 'format_text': True, 'dual_channel': None, 'webhook_url': None, 'webhook_status_code': None, 'webhook_auth': False, 'webhook_auth_header_name': None, 'speed_boost': False, 'auto_highlights_result': None, 'auto_highlights': True, 'audio_start_from': None, 'audio_end_at': None, 'word_boost': [], 'boost_param': None, 'filter_profanity': False, 'redact_pii': True, 'redact_pii_audio': False, 'redact_pii_audio_quality': 'mp3', 'redact_pii_policies': ['drug', 'injury', 'person_name'], 'redact_pii_sub': 'hash', 'speaker_labels': False, 'content_safety': True, 'iab_categories': True, 'content_safety_labels': {}, 'iab_categories

## Make Polling Endpoint

In [31]:
def make_polling_endpoint(transcript_id):
    # If upload response is input rather than raw upload_url string
    if type(transcript_id) is dict:
        transcript_id = transcript_id['id']
        
    polling_endpoint = "https://api.assemblyai.com/v2/transcript/" + transcript_id
    return polling_endpoint

In [32]:
polling_endpoint = make_polling_endpoint(transcript_request_response)

In [33]:
polling_endpoint

'https://api.assemblyai.com/v2/transcript/owke4eku0d-8ac7-4729-bdb9-1a42fcf6f70b'

In [35]:
# so dont have to rerun transcript if restart notebook
polling_endpoint = "https://api.assemblyai.com/v2/transcript/owke4eku0d-8ac7-4729-bdb9-1a42fcf6f70b"

## Wait for Completion

In [36]:
def wait_for_completion(polling_endpoint, header):
    while True:
        polling_response = requests.get(polling_endpoint, headers=header)
        polling_response = polling_response.json()

        if polling_response['status'] == 'completed':
            break
        elif polling_response['status'] == 'error':
            raise Exception(f"Error: {polling_response['error']}")

        time.sleep(5)

In [37]:
wait_for_completion(polling_endpoint, header)

In [38]:
polling_response = requests.get(polling_endpoint, headers=header)
polling_response.json()['status']

'completed'

## Fetch Results

In [45]:
with open('response.json', 'w') as f:
    f.write(json.dumps(requests.get(polling_endpoint, headers=header).json()))

In [14]:
# Get the paragraphs of the transcript
def get_paragraphs(polling_endpoint, header):
    paragraphs_response = requests.get(polling_endpoint + "/paragraphs", headers=header)
    paragraphs_response = paragraphs_response.json()

    paragraphs = []
    for para in paragraphs_response['paragraphs']:
        paragraphs.append(para)

    return paragraphs

get_paragraphs(polling_endpoint, header)

In [16]:
# Just to see what else is here
paragraphs_response = requests.get(polling_endpoint + "/paragraphs", headers=header)
paragraphs_response = paragraphs_response.json()

# Testing Stuff

## IAB Categories response

In [46]:
j = json.loads("response.json")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [130]:
dic = {
      "Travel>TravelLocations>AfricaTravel": 1.0,
      "Travel>TravelLocations>EuropeTravel": 0.7229765057563782,
      "NewsAndPolitics>Politics": 0.22560375928878784,
      "Religion&Spirituality>Christianity": 0.17373540997505188,
      "NewsAndPolitics>Law": 0.10182195901870728,
      "EventsAndAttractions>PoliticalEvent": 0.09067431092262268,
      "FamilyAndRelationships>MarriageAndCivilUnions": 0.05113469809293747,
      "Travel>TravelLocations>NorthAmericaTravel": 0.04764898121356964,
      "NewsAndPolitics>Politics>PoliticalIssues": 0.03767641261219978,
      "NewsAndPolitics>Politics>WarAndConflicts": 0.025518830865621567,
      "Travel>TravelLocations>AustraliaAndOceaniaTravel": 0.023581648245453835,
      "Hobbies&Interests>GenealogyAndAncestry": 0.013975286856293678,
      "Religion&Spirituality>Judaism": 0.012308650650084019,
      "NewsAndPolitics>Crime": 0.012244840152561665,
      "Science>Geography": 0.010910360142588615,
      "BooksAndLiterature>Biographies": 0.010122659616172314,
      "Technology&Computing>Computing>Internet": 0.00969504751265049,
      "PersonalFinance>HomeUtilities>InternetServiceProviders": 0.00960697140544653,
      "Religion&Spirituality": 0.008793771266937256,
      "NewsAndPolitics>Politics>Elections": 0.008565575815737247
    }

c = list(dic.keys())

In [131]:
c.sort()

In [132]:
c

['BooksAndLiterature>Biographies',
 'EventsAndAttractions>PoliticalEvent',
 'FamilyAndRelationships>MarriageAndCivilUnions',
 'Hobbies&Interests>GenealogyAndAncestry',
 'NewsAndPolitics>Crime',
 'NewsAndPolitics>Law',
 'NewsAndPolitics>Politics',
 'NewsAndPolitics>Politics>Elections',
 'NewsAndPolitics>Politics>PoliticalIssues',
 'NewsAndPolitics>Politics>WarAndConflicts',
 'PersonalFinance>HomeUtilities>InternetServiceProviders',
 'Religion&Spirituality',
 'Religion&Spirituality>Christianity',
 'Religion&Spirituality>Judaism',
 'Science>Geography',
 'Technology&Computing>Computing>Internet',
 'Travel>TravelLocations>AfricaTravel',
 'Travel>TravelLocations>AustraliaAndOceaniaTravel',
 'Travel>TravelLocations>EuropeTravel',
 'Travel>TravelLocations>NorthAmericaTravel']

In [193]:
def make_tree(c, ukey=''):
    # Create empty dict for current sublist
    d = dict()
    # If leaf, return None (alternative behavior: get rid of ukey and return None for if c is None)
    if c is None and ukey is None:
        return None
    elif c is None:
        return {None: None}
    else:
        for n, i in enumerate(c):
            # For topics with sublist e.g. if ['NewsAndPolitics' 'Politics'] and ['NewsAndPolitics' 'Politics', 'Elections] are both in list - need way to signify politics itself included
            if i is None:
                d[None] = None
            # If next subtopic not in dict, add it. If the remaining list empty, make value None
            elif i[0] not in d.keys():
                topic = i.pop(0)
                d[topic] = None if i == [] else [i]
            # If subtopic already in dict
            else:
                # If the value for this subtopic is only None (i.e. subject itself is a leaf), then append sublist
                if d[i[0]] is None:
                    d[i[0]] = [None, i[1:]]
                # If value for this subtopic is a list itself, then append the remaining list
                else:
                    d[i[0]].append(i[1:])
        # Recurse on remaining leaves
        for key in d:
            d[key] = make_tree(d[key], key)
    return d

In [135]:
c_orig = c

['BooksAndLiterature>Biographies',
 'EventsAndAttractions>PoliticalEvent',
 'FamilyAndRelationships>MarriageAndCivilUnions',
 'Hobbies&Interests>GenealogyAndAncestry',
 'NewsAndPolitics>Crime',
 'NewsAndPolitics>Law',
 'NewsAndPolitics>Politics',
 'NewsAndPolitics>Politics>Elections',
 'NewsAndPolitics>Politics>PoliticalIssues',
 'NewsAndPolitics>Politics>WarAndConflicts',
 'PersonalFinance>HomeUtilities>InternetServiceProviders',
 'Religion&Spirituality',
 'Religion&Spirituality>Christianity',
 'Religion&Spirituality>Judaism',
 'Science>Geography',
 'Technology&Computing>Computing>Internet',
 'Travel>TravelLocations>AfricaTravel',
 'Travel>TravelLocations>AustraliaAndOceaniaTravel',
 'Travel>TravelLocations>EuropeTravel',
 'Travel>TravelLocations>NorthAmericaTravel']

In [194]:
c = [i.split(">") for i in c_orig]

In [195]:
c

[['BooksAndLiterature', 'Biographies'],
 ['EventsAndAttractions', 'PoliticalEvent'],
 ['FamilyAndRelationships', 'MarriageAndCivilUnions'],
 ['Hobbies&Interests', 'GenealogyAndAncestry'],
 ['NewsAndPolitics', 'Crime'],
 ['NewsAndPolitics', 'Law'],
 ['NewsAndPolitics', 'Politics'],
 ['NewsAndPolitics', 'Politics', 'Elections'],
 ['NewsAndPolitics', 'Politics', 'PoliticalIssues'],
 ['NewsAndPolitics', 'Politics', 'WarAndConflicts'],
 ['PersonalFinance', 'HomeUtilities', 'InternetServiceProviders'],
 ['Religion&Spirituality'],
 ['Religion&Spirituality', 'Christianity'],
 ['Religion&Spirituality', 'Judaism'],
 ['Science', 'Geography'],
 ['Technology&Computing', 'Computing', 'Internet'],
 ['Travel', 'TravelLocations', 'AfricaTravel'],
 ['Travel', 'TravelLocations', 'AustraliaAndOceaniaTravel'],
 ['Travel', 'TravelLocations', 'EuropeTravel'],
 ['Travel', 'TravelLocations', 'NorthAmericaTravel']]

In [197]:
m = make_tree(c, '')

In [183]:
m

{'BooksAndLiterature': {'Biographies': None},
 'EventsAndAttractions': {'PoliticalEvent': None},
 'FamilyAndRelationships': {'MarriageAndCivilUnions': None},
 'Hobbies&Interests': {'GenealogyAndAncestry': None},
 'NewsAndPolitics': {'Crime': None,
  'Law': None,
  'Politics': {None: None,
   'Elections': None,
   'PoliticalIssues': None,
   'WarAndConflicts': None}},
 'PersonalFinance': {'HomeUtilities': {'InternetServiceProviders': None}},
 'Religion&Spirituality': {None: None, 'Christianity': None, 'Judaism': None},
 'Science': {'Geography': None},
 'Technology&Computing': {'Computing': {'Internet': None}},
 'Travel': {'TravelLocations': {'AfricaTravel': None,
   'AustraliaAndOceaniaTravel': None,
   'EuropeTravel': None,
   'NorthAmericaTravel': None}}}

In [198]:
m

{'BooksAndLiterature': {'Biographies': {None: None}},
 'EventsAndAttractions': {'PoliticalEvent': {None: None}},
 'FamilyAndRelationships': {'MarriageAndCivilUnions': {None: None}},
 'Hobbies&Interests': {'GenealogyAndAncestry': {None: None}},
 'NewsAndPolitics': {'Crime': {None: None},
  'Law': {None: None},
  'Politics': {None: None,
   'Elections': {None: None},
   'PoliticalIssues': {None: None},
   'WarAndConflicts': {None: None}}},
 'PersonalFinance': {'HomeUtilities': {'InternetServiceProviders': {None: None}}},
 'Religion&Spirituality': {None: None,
  'Christianity': {None: None},
  'Judaism': {None: None}},
 'Science': {'Geography': {None: None}},
 'Technology&Computing': {'Computing': {'Internet': {None: None}}},
 'Travel': {'TravelLocations': {'AfricaTravel': {None: None},
   'AustraliaAndOceaniaTravel': {None: None},
   'EuropeTravel': {None: None},
   'NorthAmericaTravel': {None: None}}}}

In [100]:
type({}) is dict

True

In [65]:
dict()

{}

# Gradio App Demo

In [1]:
%%capture

!pip install gradio

In [ ]:
# virtualenv venv
# .\venv\Scripts\activate.bat
# pip install jupyter
# ipython kernel install --name "gradio-venv" --user

In [4]:
API_KEY = "a6f53d3fe7f542be8ce148432f0ff9a0"

In [3]:
import gradio as gr

In [6]:
def greet(name):
    return "hello"+name

with gr.Blocks(css="body {background-color: red}") as demo:
    name = gr.Textbox()
    output = gr.Textbox()
    greet_btn = gr.Button()
    greet_btn.click(greet, name, output)
    
demo.launch()

Running on local URL:  http://127.0.0.1:7861/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x211296a7640>, 'http://127.0.0.1:7861/', None)

In [7]:
import gradio as gr

scores = []

def track_score(score):
    scores.append(score)
    top_scores = sorted(scores, reverse=True)[:3]
    return top_scores

demo = gr.Interface(
    track_score, 
    gr.Number(label="Score"), 
    gr.JSON(label="Top Scores")
)
demo.launch()

Running on local URL:  http://127.0.0.1:7862/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x2112972f1f0>, 'http://127.0.0.1:7862/', None)

In [8]:
import random
import gradio as gr

def chat(message, history):
    history = history or []
    message = message.lower()
    if message.startswith("how many"):
        response = random.randint(1, 10)
    elif message.startswith("how"):
        response = random.choice(["Great", "Good", "Okay", "Bad"])
    elif message.startswith("where"):
        response = random.choice(["Here", "There", "Somewhere"])
    else:
        response = "I don't know"
    history.append((message, response))
    return history, history

chatbot = gr.Chatbot().style(color_map=("green", "pink"))
demo = gr.Interface(
    chat,
    ["text", "state"],
    [chatbot, "state"],
    allow_flagging="never",
)
demo.launch()


Running on local URL:  http://127.0.0.1:7863/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x211298b30a0>, 'http://127.0.0.1:7863/', None)

In [82]:
import gradio as gr
import random

secret_word = "gradio"

with gr.Blocks() as demo:    
    used_letters_var = gr.State([])
    with gr.Row() as row:
        with gr.Column():
            input_letter = gr.Textbox(label="Enter letter")
            btn = gr.Button("Guess Letter")
        with gr.Column():
            hangman = gr.Textbox(
                label="Hangman",
                value="_"*len(secret_word)
            )
            used_letters_box = gr.Textbox(label="Used Letters")

    def guess_letter(letter, used_letters):
        used_letters.append(letter)
        answer = "".join([
            (letter if letter in used_letters else "_")
            for letter in secret_word
        ])
        return {
            used_letters_var: used_letters,
            used_letters_box: ", ".join(used_letters),
            hangman: answer
        }
    btn.click(
        guess_letter, 
        [input_letter, used_letters_var],
        [used_letters_var, used_letters_box, hangman]
        )
demo.launch()

AttributeError: module 'gradio' has no attribute 'State'

In [85]:
with gr.Blocks() as demo:
    with gr.Tab("Lion"):
        gr.Image("lion.jpg")
        gr.Button("New Lion")
    with gr.Tab("Tiger"):
        gr.Image("tiger.jpg")
        gr.Button("New Tiger")
        
demo.launch()

AttributeError: module 'gradio' has no attribute 'Tab'

# Audio Intelligence Dashboard

In [30]:
import gradio as gr

def chg_file_type(choice):
    if choice == "Local File":
        return gr.Textbox(label="prompt 1234")
    elif choice == "Remote File":
        return gr.Textbox(label="prompt 1234234")

    def txt_chg():
        return gr.update()

with gr.Blocks() as demo:
    with gr.Row():
        text11 = gr.Textbox(label="prompt 1")
        text12 = gr.Textbox(label="prompt 1")
        
        file_type = gr.Radio(choices=["Local File", "Remote File"], value="Local File", interactive=True)
        file_type.change(fn=chg_file_type, inputs=None, outputs=text11)
        
    text21 = gr.HTML("<p>this is some text</p>")
        
    with gr.Row():
        with gr.Column():
            text1 = gr.Textbox(label="prompt 1")
            text2 = gr.Textbox(label="prompt 2")
        with gr.Column():
            img1 = gr.Image("https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Image_created_with_a_mobile_phone.png/800px-Image_created_with_a_mobile_phone.png")
            btn = gr.Button("Go").style(full_width=True)
demo.launch()

Running on local URL:  http://127.0.0.1:7881/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x2112c82e1a0>, 'http://127.0.0.1:7881/', None)

In [32]:
import gradio as gr

def change_textbox(choice):
    if choice == "short":
        return gr.update(lines=2, visible=True, value="Short story: ")
    elif choice == "long":
        return gr.update(lines=8, visible=True, value="Long story...")
    else:
        return gr.update(visible=False)

with gr.Blocks() as demo:
    gr.Markdown(
    """
    # Hello World!
    Start typing below to see the output.
    """)
    inp = gr.Textbox(placeholder="What is your name?")
    out = gr.Textbox()
    inp.change(welcome, inp, out)

demo.launch()


Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Running on local URL:  http://127.0.0.1:7884/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x2112c9e2140>, 'http://127.0.0.1:7884/', None)

In [79]:
# Blocks Example
counter = 1

class OtherTextbox(gr.Textbox):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    

def change_audio_source(val):
    global counter
    counter += 1
    if counter % 2 == 0:
        return [gr.Audio.update(visible=False), gr.Image.update(visible=True)]
    else:
        return [gr.Audio.update(visible=True), gr.Image.update(visible=False)]
    
def fn2(val):
    return val
    

import gradio as gr
with gr.Blocks() as demo:
    radio = gr.Radio([1, 2, 4], label="Set the value of the number")
    with gr.Box():
        audio_file = gr.Audio(interactive=True)
        mic_recording = gr.Audio(source="microphone", visible=False, interactive=True)
        
    number = gr.Textbox()
    #audio_wave = gr.Plot()
    
    
    radio.change(fn=change_audio_source, inputs=radio, outputs=[audio_file, mic_recording])
    for component in [audio_file, mic_recording]:
        getattr(component, 'change')(fn=fn2, inputs=component, outputs=number)
    
    
demo.launch()

Running on local URL:  http://127.0.0.1:7921/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x211304a6da0>, 'http://127.0.0.1:7921/', None)